In [1]:
import os

import pandas as pd
import numpy as np

In [3]:
tmall = pd.read_csv('../../data/tmall/ijcai2016_taobao.csv')

In [5]:
tmall.act_ID.value_counts()

0    35179371
1     9348756
Name: act_ID, dtype: int64

In [4]:
tmall = tmall[['use_ID', 'ite_ID', 'act_ID', 'time']]

In [5]:
max_session_type = tmall.groupby(['use_ID', 'ite_ID'])['act_ID'].max().rename('max_type')

In [6]:
tmall = tmall.join(max_session_type, ['use_ID', 'ite_ID'])

In [7]:
tmall = tmall[tmall['act_ID'] == tmall['max_type']]

In [8]:
tmall.reset_index(drop=True, inplace=True)
tmall.drop(columns='max_type', inplace=True)

In [9]:
click_weight, purchase_weight = \
    tmall.shape[0] / tmall['act_ID'].value_counts()

In [10]:
tmall = tmall.join(
    tmall.groupby(['use_ID', 'ite_ID'])['act_ID'].count().rename('count'),
    on=['use_ID', 'ite_ID']
)

In [11]:
tmall.drop_duplicates(['use_ID', 'ite_ID'], inplace=True)

In [12]:
click_border = click_weight * (purchase_weight // click_weight)

In [13]:
tmall['weight'] = np.where(
    tmall['ite_ID'] == 0,
    np.minimum(click_border, tmall['count'] * click_weight),
    tmall['count'] * purchase_weight
)

In [14]:
tmall.drop(columns=['act_ID', 'count'], inplace=True)

In [15]:
tmall.time = pd.to_datetime(tmall.time, format='%Y%m%d').astype(int)

In [17]:
tmall.to_parquet(os.path.join('..', '..', 'data', 'tmall', 'interactions.parquet'), index=False)